<a href="https://colab.research.google.com/github/anna-alt/AI-Lab/blob/main/SceneRecognition1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler, ConcatDataset
from torchvision import datasets, transforms
from torchvision import models

import numpy as np
import sklearn



In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device

device(type='cuda')

In [3]:
trainset_size = 5000
val_size = 500
num_epochs = 10
image_size = (224,224)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Resize(image_size),
                              
                              ])
# Download and load the training data
trainset = datasets.SUN397('/content/drive/MyDrive/Data', download= False, transform=transform)
splittrain = torch.utils.data.random_split(trainset, [trainset_size, len(trainset) - trainset_size])[0]
train_loader = DataLoader(splittrain, batch_size=32,num_workers = 2, shuffle=True)

validationset = datasets.SUN397("/content/drive/MyDrive/Data", download=False, transform=transform)
splitvalidation = torch.utils.data.random_split(validationset, [val_size, len(trainset) - val_size])[0]
val_loader = DataLoader(splitvalidation, batch_size=32,num_workers = 2, shuffle=True)



In [ ]:
for image, label in train_loader:
  labels = np.array(image)
  numpy_labels = label.numpy()
  print(labels)
  break


In [ ]:
model = models.resnet50(weights = "DEFAULT")
model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8)
criterion = nn.CrossEntropyLoss()
print(model)

In [8]:
def train(epoch, log_interval=20):
    # Set model to training mode
    model.train()
    
    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(train_loader):
        # Copy data to GPU if needed
        data = data.to(device)
        target = target.to(device)

        # Zero gradient buffers
        optimizer.zero_grad() 
        
        # Pass data through the network
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

In [9]:

def validate(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in val_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(val_loader)
    loss_vector.append(val_loss)

    print(pred)

    accuracy = 100. * correct.to(torch.float32) / len(val_loader.dataset)
    accuracy_vector.append(accuracy)

    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_loader.dataset), accuracy))

In [10]:
def Predic(loss_vector,accuracy_vector):
    val_loss, correct = 0, 0
    for data, target in val_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
    print( pred)
    print("/n")
    print(target)


In [11]:
lossv, accv = [], []
for epoch in range(1, num_epochs + 1):
    train(epoch)
    validate(lossv, accv)
    Predic(lossv,accv)

Train Epoch: 1 [0/5000 (0%)]	Loss: 7.627412
Train Epoch: 1 [640/5000 (13%)]	Loss: 6.729174
Train Epoch: 1 [1280/5000 (25%)]	Loss: 6.642689
Train Epoch: 1 [1920/5000 (38%)]	Loss: 5.491829
Train Epoch: 1 [2560/5000 (51%)]	Loss: 5.561943
Train Epoch: 1 [3200/5000 (64%)]	Loss: 5.538623


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385478 bytes but only got 23143. Skipping tag 513
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7276331008 bytes but only got 0. Skipping tag 7
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1879769088 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3014066176 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3568893952 bytes but only got 23144. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-pac

Train Epoch: 1 [3840/5000 (76%)]	Loss: 5.552651
Train Epoch: 1 [4480/5000 (89%)]	Loss: 4.719298
tensor([ 46,   2, 193, 133,  63,   2,  24, 234, 193, 376, 227, 338, 264, 892,
        245,  60, 300,   2, 184,  85], device='cuda:0')

Validation set: Average loss: 4.6550, Accuracy: 97/500 (19%)

tensor([370,  46, 245,  82, 300, 213, 227, 245, 213,  46,  60, 118,  75, 245,
        174, 234, 279, 335,   2, 937], device='cuda:0')
/n
tensor([284,  46, 245,  77, 208, 131,  58,  46, 213,  52,  60, 117, 391, 256,
         52, 106,  53,   6, 145,  57], device='cuda:0')
Train Epoch: 2 [0/5000 (0%)]	Loss: 3.725392
Train Epoch: 2 [640/5000 (13%)]	Loss: 3.766303
Train Epoch: 2 [1280/5000 (25%)]	Loss: 3.606399
Train Epoch: 2 [1920/5000 (38%)]	Loss: 3.531874
Train Epoch: 2 [2560/5000 (51%)]	Loss: 3.267872
Train Epoch: 2 [3200/5000 (64%)]	Loss: 3.587355
Train Epoch: 2 [3840/5000 (76%)]	Loss: 3.627251
Train Epoch: 2 [4480/5000 (89%)]	Loss: 2.901611


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385478 bytes but only got 23143. Skipping tag 513
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7276331008 bytes but only got 0. Skipping tag 7
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1879769088 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3014066176 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3568893952 bytes but only got 23144. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-pac

tensor([206,  81,  60, 300, 103, 193, 361, 163,  69, 120, 213, 133,  85, 318,
        349, 133, 200, 140, 359, 234], device='cuda:0')

Validation set: Average loss: 3.3271, Accuracy: 181/500 (36%)

tensor([200,  55, 342, 359, 100,  48, 331,  81, 107, 119,  99, 213, 338, 136,
        300, 157, 225,   6, 103, 213], device='cuda:0')
/n
tensor([200, 221, 342,   5, 100, 192, 331, 298, 122, 168, 392, 315, 217,  59,
        300, 157, 110,   6, 103, 213], device='cuda:0')
Train Epoch: 3 [0/5000 (0%)]	Loss: 2.290776
Train Epoch: 3 [640/5000 (13%)]	Loss: 1.110156
Train Epoch: 3 [1280/5000 (25%)]	Loss: 1.680406
Train Epoch: 3 [1920/5000 (38%)]	Loss: 1.711351


...

Train Epoch: 9 [2560/5000 (51%)]	Loss: 0.014011
Train Epoch: 9 [3200/5000 (64%)]	Loss: 0.015880
Train Epoch: 9 [3840/5000 (76%)]	Loss: 0.009617
Train Epoch: 9 [4480/5000 (89%)]	Loss: 0.012644
tensor([115, 103, 216, 376,  81, 133,  92, 213, 232, 225, 227, 192, 245, 213,
        370,  96, 293, 111, 358,  85], device='cuda:0')

Validation set: Average loss: 2.5677, Accuracy: 245/500 (49%)

tensor([  8,  48, 213, 245, 270, 199, 225,  48, 184, 124, 280, 136, 253, 384,
         47, 227,  60, 193, 170, 110], device='cuda:0')
/n
tensor([  8,  48,  41, 328,  19,  75, 382,  93, 256,  26, 280, 136, 278, 383,
        227,  48,  60, 203, 170, 110], device='cuda:0')
Train Epoch: 10 [0/5000 (0%)]	Loss: 0.006765
Train Epoch: 10 [640/5000 (13%)]	Loss: 0.013535
Train Epoch: 10 [1280/5000 (25%)]	Loss: 0.013876
Train Epoch: 10 [1920/5000 (38%)]	Loss: 0.010377
Train Epoch: 10 [2560/5000 (51%)]	Loss: 0.010870
Train Epoch: 10 [3200/5000 (64%)]	Loss: 0.018206


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385478 bytes but only got 23143. Skipping tag 513
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7276331008 bytes but only got 0. Skipping tag 7
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1879769088 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3014066176 bytes but only got 0. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3568893952 bytes but only got 23144. Skipping tag 4
  warnings.warn(
/usr/local/lib/python3.8/dist-pac

Train Epoch: 10 [3840/5000 (76%)]	Loss: 0.008270


/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 541786190 bytes but only got 0. Skipping tag 16576
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2256669952 bytes but only got 0. Skipping tag 17
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 67518482 bytes but only got 0. Skipping tag 7
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4454482224 bytes but only got 0. Skipping tag 237
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:767: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2359296 bytes but only got 0. Skipping tag 4096
  warnings.warn(
/usr/local/lib/python3.8/dist-packag

Train Epoch: 10 [4480/5000 (89%)]	Loss: 0.008527
tensor([ 21, 379, 245, 184, 359, 297,  85, 319,  47, 103,  26, 338, 161,  34,
         22, 146, 227, 151, 143,  21], device='cuda:0')

Validation set: Average loss: 2.6374, Accuracy: 236/500 (47%)

tensor([101,   6, 170, 300,  86, 200, 232,  41,  47, 216,  47,  34, 105,   5,
         10, 375,  41, 213, 359, 100], device='cuda:0')
/n
tensor([ 44,   6, 170, 128, 392, 200, 193,  41, 177, 216,  47, 300, 105, 298,
         10, 375, 213, 213,   5, 100], device='cuda:0')


In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))